# QKD protocols:

Change as required and create pull requests.  
Here's what we need to do:  
Basics:  
1) Ideal QKD Protocol, including verification steps.  
2) Introduce practical classical notions such as error correction and hashes, etc as required.  
3) Analysis of performance.  
  
Later on:  
1) Proper network archeticture, establishing entanglement between nodes by swapping and distillation.

Add stuff to this list and for each thing/module, open an issue. Reference the issue when you commit and make a pull request.

## Getting started  
Lets initialize required packages and modules

In [4]:
# Initializations
import numpy as np
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, Aer
from qiskit.tools.visualization import circuit_drawer, plot_histogram

## Basic Quantum Key Distribution protocol
Bleh